In [1]:
!pip install psycopg2-binary



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 3.7 MB/s eta 0:00:0000:0100:01


In [1]:
from peewee import Model, PostgresqlDatabase, CharField, UUIDField, IntegerField, SQL
import uuid
from uuid import UUID
db = PostgresqlDatabase('parking-tracker', user='postgres', password='postgres', host='localhost', port=5432)

class Parkings(Model):
    id = IntegerField(primary_key=True)  # Serial in PostgreSQL is typically represented as an IntegerField in Peewee
    display_id = UUIDField(default=uuid.uuid4, unique=True, index=True, null=False)
    latitude = CharField(max_length=100)
    longitude = CharField(max_length=100)
    polygon_axes = CharField(max_length = 100)
    curr_motor = IntegerField(default=0)
    max_space = IntegerField(default=0)
    class Meta:
        database = db
        table_name = 'parkings'
    def __str__(self):
        return f"Parking(\nid={self.id}, \ndisplay_id={self.display_id}, \nlatitude={self.latitude}, \nlongitude={self.longitude}, \ncurr_motor={self.curr_motor}, \nmax_space={self.max_space})"
        

db.connect()
# db.create_tables([Parkings])


True

In [2]:
# Inserting new instance of parking
new_parking = Parkings.create(
  latitude = '25.01370607644918',
  longitude = '121.53468199176018',
  curr_motor = 2,
  max_space = 10,
)



In [28]:
print(new_parking.display_id)

ba16a376-a4bc-48aa-84cf-3b65e666e976


In [54]:
# Updating table
Parkings.update(curr_motor = 5).where(Parkings.display_id == 'ba16a376-a4bc-48aa-84cf-3b65e666e976').execute()


1

In [69]:
# Get data from table
import ast
parking = Parkings.get(Parkings.display_id == '712b78b3-28dd-404d-b0b2-232628097ba6')
str_polygon_axes = parking.polygon_axes
if str_polygon_axes != None:
  polygons_axes = ast.literal_eval(parking.polygon_axes)
  print(parking.polygon_axes)

[[1,1],[2,2],[3,3]]


In [66]:
#Create or Update if created
def create_or_get_parking( display_id: uuid.UUID, latitude: str = None, longitude: str = None, curr_motor: int = 0, max_space: int = 0):
    defaults = dict()
    if longitude != None or longitude != None: #Create Parking
      defaults = {'display_id':display_id, 'latitude': latitude, 'longitude': longitude, 'curr_motor':curr_motor}
    else: #Update Parking
      defaults = {'curr_motor': curr_motor}
    parking, created = Parkings.get_or_create(
      display_id=display_id,
      defaults=defaults)
    return parking

In [67]:
from update_db import create_or_get_parking
parking_data = {'state': 'Registering', 'uuid': 'e48dbd94-cd7a-42ec-b93c-da68c9f1be17', 'latitude': None, 'longitude': None, 'curr_motor': 0}
state = parking_data['state']
uuid = parking_data['uuid']
latitude = parking_data['latitude']
longitude = parking_data['longitude']
curr_motor = parking_data['curr_motor']

create_or_get_parking(uuid, latitude, longitude, curr_motor)

<Parkings: Parking(
id=11, 
display_id=e48dbd94-cd7a-42ec-b93c-da68c9f1be17, 
latitude=None, 
longitude=None, 
curr_motor=0, 
max_space=0)>

In [ ]:
#